In [29]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch_geometric.nn import MessagePassing
from torch_geometric.utils import add_self_loops, degree

from torch_geometric.data import Data
from torch_geometric.data import DataLoader

from torch_geometric.datasets import KarateClub

# Define a simple Graph Neural Network (GNN) layer
class GNNLayer(MessagePassing):
    def __init__(self, in_channels, out_channels):
        super(GNNLayer, self).__init__(aggr="add")
        self.lin = nn.Linear(in_channels, out_channels)

    def forward(self, x, edge_index):
        # Add self-loops to the adjacency matrix
        edge_index, _ = add_self_loops(edge_index, num_nodes=x.size(0))

        # Apply a linear transformation to node features
        x = self.lin(x)

        # Propagate messages through the graph
        return self.propagate(edge_index, x=x)

    def message(self, x_j):
        # Messages are just the node features of the neighboring nodes
        return x_j

# Define a simple GNN model
class GNNModel(nn.Module):
    def __init__(self, in_channels, hidden_channels, out_channels):
        super(GNNModel, self).__init__()
        self.conv1 = GNNLayer(in_channels, hidden_channels)
        self.conv2 = GNNLayer(hidden_channels, out_channels)

    def forward(self, x, edge_index):
        x = F.relu(self.conv1(x, edge_index))
        x = self.conv2(x, edge_index)
        return x

# Generate a synthetic graph dataset (Karate Club dataset)
dataset = KarateClub()
data = dataset[0]  # Get the first graph in the dataset


In [30]:
data

Data(x=[34, 34], edge_index=[2, 156], y=[34], train_mask=[34])

In [31]:
from torch_geometric.utils import negative_sampling

# Create positive edges based on the existing edges in the graph
pos_edge_index = dataset.data.edge_index

# Create negative edges by randomly sampling pairs of unconnected nodes
num_neg_samples = pos_edge_index.size(1)  # Number of negative samples equal to positive samples
neg_edge_index = negative_sampling(pos_edge_index, num_nodes=dataset.data.num_nodes, num_neg_samples=num_neg_samples)


In [32]:

from sklearn.metrics import accuracy_score
# Define the GNN model
model = GNNModel(in_channels=dataset.num_features, hidden_channels=64, out_channels=1)

# Define a binary classification loss (e.g., Binary Cross-Entropy)
criterion = nn.BCEWithLogitsLoss()

# Define an optimizer
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)

# Split the edges into positive (existing) and negative (non-existing) edges
pos_edge_index, neg_edge_index = neg_edge_index, pos_edge_index

# Training loop
for epoch in range(100):
    optimizer.zero_grad()

    # Predict link directions for positive and negative edges
    pos_pred = model(dataset.data.x, pos_edge_index).squeeze()
    neg_pred = model(dataset.data.x, neg_edge_index).squeeze()

    # Concatenate positive and negative predictions
    all_pred = torch.cat([pos_pred, neg_pred], dim=0)

    # Create target labels (1 for positive edges, 0 for negative edges)
    labels = torch.cat([torch.ones(pos_pred.size(0)), torch.zeros(neg_pred.size(0))], dim=0)

    # Calculate the binary classification loss
    loss = criterion(all_pred, labels)

    # Backpropagation
    loss.backward()
    optimizer.step()

    if epoch % 10 == 0:
        print(f'Epoch {epoch}, Loss: {loss.item()}')

# Evaluate the model on the validation set
with torch.no_grad():
    val_pos_pred = model(dataset.data.x, pos_edge_index).squeeze()
    val_neg_pred = model(dataset.data.x, neg_edge_index).squeeze()

    val_all_pred = torch.cat([val_pos_pred, val_neg_pred], dim=0)
    val_labels = torch.cat([torch.ones(val_pos_pred.size(0)), torch.zeros(val_neg_pred.size(0))], dim=0)

    # Convert predictions to binary values (0 or 1)
    val_pred_labels = (val_all_pred > 0).float()

    # Calculate accuracy on the validation set
    val_accuracy = accuracy_score(val_labels.cpu().numpy(), val_pred_labels.cpu().numpy())
    print(f'Validation Accuracy: {val_accuracy * 100:.2f}%')

Epoch 0, Loss: 0.9028078317642212
Epoch 10, Loss: 0.23769505321979523
Epoch 20, Loss: 0.07001117616891861
Epoch 30, Loss: 0.023156002163887024
Epoch 40, Loss: 0.009636993519961834
Epoch 50, Loss: 0.004885855596512556
Epoch 60, Loss: 0.0030313164461404085
Epoch 70, Loss: 0.0021121962927281857
Epoch 80, Loss: 0.0016226728912442923
Epoch 90, Loss: 0.001309235580265522
Validation Accuracy: 100.00%


In [33]:
node_1 = 10
dataset.data.x[node_1].unsqueeze(0)

f:\CODE\ParisTennis\env\Lib\site-packages\torch_geometric\data\in_memory_dataset.py:189: UserWarning: It is not recommended to directly access the internal storage format `data` of an 'InMemoryDataset'. If you are absolutely certain what you are doing, access the internal storage via `InMemoryDataset._data` instead to suppress this warning. Alternatively, you can access stacked individual attributes of every graph via `dataset.{attr_name}`.
  warnings.warn(msg)


tensor([[0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]])

In [57]:
data.edge_index

tensor([[ 0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  1,  1,
          1,  1,  1,  1,  1,  1,  1,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  3,
          3,  3,  3,  3,  3,  4,  4,  4,  5,  5,  5,  5,  6,  6,  6,  6,  7,  7,
          7,  7,  8,  8,  8,  8,  8,  9,  9, 10, 10, 10, 11, 12, 12, 13, 13, 13,
         13, 13, 14, 14, 15, 15, 16, 16, 17, 17, 18, 18, 19, 19, 19, 20, 20, 21,
         21, 22, 22, 23, 23, 23, 23, 23, 24, 24, 24, 25, 25, 25, 26, 26, 27, 27,
         27, 27, 28, 28, 28, 29, 29, 29, 29, 30, 30, 30, 30, 31, 31, 31, 31, 31,
         31, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 33, 33, 33, 33, 33,
         33, 33, 33, 33, 33, 33, 33, 33, 33, 33, 33, 33],
        [ 1,  2,  3,  4,  5,  6,  7,  8, 10, 11, 12, 13, 17, 19, 21, 31,  0,  2,
          3,  7, 13, 17, 19, 21, 30,  0,  1,  3,  7,  8,  9, 13, 27, 28, 32,  0,
          1,  2,  7, 12, 13,  0,  6, 10,  0,  6, 10, 16,  0,  4,  5, 16,  0,  1,
          2,  3,  0,  2, 30, 32, 33,  2, 33,  0,  4

In [40]:
dataset.data.x[10]

tensor([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.])

In [48]:
data.validate()

True

In [65]:
# Specify the nodes you want to predict an edge for (e.g., node 5 and node 10)
node_5 = 5
node_10 = 10

# Prepare input features for both nodes
node_5_features = dataset.data.x[node_5].unsqueeze(0)  # Shape: [1, num_features]
node_10_features = dataset.data.x[node_10].unsqueeze(0)  # Shape: [1, num_features]

# Create an edge tensor representing the edge between node 5 and node 10
edge_tensor = torch.tensor([[node_5, node_10]], dtype=torch.long).t()

# Predict the direction of the edge
with torch.no_grad():
    prediction = model(torch.cat([node_5_features, node_10_features], dim=0), edge_tensor)


IndexError: Encountered an index error. Please ensure that all indices in 'edge_index' point to valid indices in the interval [0, 1] (got interval [0, 5])

In [63]:

if prediction[0] > 0:
    print(f"Predicted edge direction: Node {node_0} to Node {node_1}")
else:
    print(f"Predicted edge direction: Node {node_1} to Node {node_0}")

Predicted edge direction: Node 10 to Node 0
